In [1]:
import sys
sys.path.append("..")
import sound_eval as sev
import param_generation as pg
from common_vars import *
import imp
imp.reload(pg)
imp.reload(sev)
import imp
from deap import base, creator
import random
from deap import tools
import scipy.spatial.distance as d
from helpers import rToParams
from IPython.display import Audio
import pandas as pd

In [34]:
#prepare dataframe
NGEN=5
STACK_SIZE=2
# df=pd.DataFrame(columns=["evo_round","pop_fitness","hof_fitness","hof_1","hof_2","hof_3"])
write_path="./experiment_data/%s-%s"%(NGEN,STACK_SIZE)
df.to_csv(write_path,index=False)
df

,evo_round,pop_fitness,hof_fitness,hof_1,hof_2,hof_3
0,0,2.607012e-04,0.047923,"(0.6036707758903503,)","(0.34862667322158813,)","(0.0026070112362504005,)"
1,1,7.832317e-17,0.047923,"(0.6036707758903503,)","(0.34862667322158813,)","(0.0026070112362504005,)"
2,2,3.535840e-13,0.047923,"(0.6036707758903503,)","(0.34862667322158813,)","(0.0026070112362504005,)"
3,3,2.059453e-13,0.047923,"(0.6036707758903503,)","(0.34862667322158813,)","(0.0026070112362504005,)"
4,4,7.946548e-17,0.047923,"(0.6036707758903503,)","(0.34862667322158813,)","(0.0026070112362504005,)"
...,...,...,...,...,...,...
145,45,4.889438e-15,0.110254,"(0.9173575043678284,)","(0.6036707758903503,)","(0.34862667322158813,)"
146,46,4.501021e-15,0.110254,"(0.9173575043678284,)","(0.6036707758903503,)","(0.34862667322158813,)"
147,47,1.203081e-10,0.110254,"(0.9173575043678284,)","(0.6036707758903503,)","(0.34862667322158813,)"
148,48,1.057643e-16,0.110254,"(0.9173575043678284,)","(0.6036707758903503,)","(0.34862667322158813,)"


In [7]:
toolbox = base.Toolbox()
STACK_SIZE = 4
POP_SIZE = 10
GREP,MUTPB = 0.5, 1
MUTENV,MUTTEXT=0.3,0.2

class drum_program():
    def __init__(self,stack_num=STACK_SIZE):
        self.program=[pg.RandomParams() for i in range(stack_num)]
    def mutate(self):
        for p in self.program:p.mutate(e=0.8,t=0.8,s=0.2)
        return self
creator.create("FitnessMin", base.Fitness, weights=(1,))
creator.create("Individual",drum_program, fitness=creator.FitnessMin)

drum="kick"

def initPopulation(pcls,ng):
    ipop=[creator.Individual()
          for x in range(ng)]
    return pcls(ipop)

def evaluate(individual):
    o=pg.ensemble(individual.program)
    return (sev.evalSound(o),)

def randSelect(l,k):
    return random.sample(l, k)

# toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list,)


toolbox.register("select_best", tools.selBest,fit_attr='fitness')
toolbox.register("select_random",randSelect)
toolbox.register("evaluate", evaluate)

hof=tools.HallOfFame(POP_SIZE*2)

/home/asalimi/miniconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/asalimi/miniconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [30]:
%%time
#what's going on here?
# initiate new population, mutate them, evaluate
def evolve(pop,hof):
    newGenes=toolbox.population_guess(int(0.25*POP_SIZE))
    offspring = pop.copy() + newGenes

    # Apply crossover and mutation on the offspring
#     for j,child in enumerate(offspring):
#         for i,program in enumerate(child.program):
#             if random.random() < GREP:                
#                 child.program[i]=random.choice(hof[0:3]).program[i]
#                 del child.fitness.values
#         offspring[j]=child
#         pass

    # Clone the selected individuals        
    for i,mutant in enumerate(offspring):
            offspring[i]=mutant.mutate()
            del offspring[i].fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    #update hof

    survivors = toolbox.select_best(offspring,POP_SIZE)
    return survivors.copy(),hof
    
NGEN=50
def main(hof):
    pop = toolbox.population_guess(POP_SIZE)
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    for g in range(NGEN):
        pop,hof=evolve(pop,hof)
        hof.update(pop)
        hof_fitness=np.average([f.fitness.values[0] for f in hof])
        pop_fitness=np.average([f.fitness.values[0] for f in pop])
        print (g,"\r",end="")
        df.loc[len(df)]=([g,pop_fitness,hof_fitness,hof[0].fitness.values,
                          hof[1].fitness.values,hof[2].fitness.values])
    return pop
pop=main(hof)
# print([i.fitness.values for i in pop])

CPU times: user 3min 20s, sys: 47.1 s, total: 4min 7s
Wall time: 59.8 s


In [27]:
k=hof[2]
print(k.program,"prog")

o=pg.ensemble(k.program)
print(evaluate(k),k.fitness.values)

Audio(o,rate=SR,autoplay=True,embed=True)

[<param_generation.RandomParams object at 0x7f4bc4846b90>, <param_generation.RandomParams object at 0x7f4af962fd50>, <param_generation.RandomParams object at 0x7f4af96fb7d0>, <param_generation.RandomParams object at 0x7f4af962f4d0>] prog
(0.20801275968551636,) (0.22420097887516022,)


In [33]:
from ipywidgets import GridspecLayout,HBox,widgets
# What did our DAEP search return?
z=hof
x,y=3,int((len(z)/4))
grid=[]
grid = GridspecLayout(x, y)
for i in range(x):
    for j in range(y):
        o= widgets.Output()
        sample_number=i+j+(i*(y-1))
        pset=z[sample_number]
        try:
            out=pg.ensemble(pset.program)
            with o:
                display(Audio(out,rate=SR,autoplay=False,embed=True),pset.fitness.values,evaluate(pset))
            grid[i, j] = o
        except:
            continue
display(grid)

(0.9173575043678284,)

(0.9134392738342285,)

(0.6036707758903503,)

(0.6036707758903503,)

(0.34862667322158813,)

...

(0.00045761658111587167,)

(0.0005554113886319101,)

(0.00010468094114912674,)

(0.00015613011782988906,)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [528]:
# z=toolbox.population_guess(2)[1]

# print(pg.ensemble(z.program))
# print(z)
# print(evaluate(z))
# for x in z.program:
#     print(x.__dict__)
#     for i in range(10):
#         x=x.mutate()
#     print(x.__dict__)

print(evaluate(z))
Audio(pg.ensemble(z),rate=SR,autoplay=True)

AttributeError: 'HallOfFame' object has no attribute 'program'